In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sys

sys.path.append('../../')
from model_comparison.utils import *
from model_comparison.mdns import Trainer, MultivariateMogMDN, PytorchMultivariateMoG

import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 20
mpl.rcParams['axes.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 12
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['figure.figsize'] = (15, 5)

%matplotlib inline

In [ ]:
# generate 2D Gaussian test data 
ntest = 10
sample_size = 1000
prior = scipy.stats.multivariate_normal(mean=[0, 0], cov=1)
thetas = prior.rvs(ntest)

In [ ]:
qis = []

for th in thetas: 
    # get x 
    model = scipy.stats.multivariate_normal(mean=th, cov=1)
    x = model.rvs(size=sample_size)
    # get posterior 
    sigma_0 = prior.cov
    mu_0 = prior.mean
    sigma = model.cov
    sigma_N = np.linalg.inv(np.linalg.inv(sigma_0) + sample_size * np.linalg.inv(sigma))
    mu_N = sigma_N.dot(sample_size * np.linalg.inv(sigma).dot(x.mean(axis=0)) + np.linalg.inv(sigma_0).dot(mu_0))
    post = scipy.stats.multivariate_normal(mean=mu_N, cov=sigma_N)
    p1 = scipy.stats.norm(loc=mu_N[0], scale=sigma_N[0, 0])
    p2 = scipy.stats.norm(loc=mu_N[1], scale=sigma_N[1, 1])
    qis.append(post.cdf(th))

In [ ]:
plt.hist(qis, bins='auto');

## Quantile distribution under the same distribution in 1D and in 2D

In [ ]:
# 1D 
fig, ax = plt.subplots(1, 2, sharex=True, figsize=(10, 3))
var1 = scipy.stats.norm(loc=0, scale=1)
samples = var1.rvs(100000)
quantiles = var1.cdf(samples)
ax[0].set_title('1D Quantile distribution')
ax[0].hist(quantiles, bins='auto');
# 2D 
var2 = scipy.stats.multivariate_normal(mean=[0, 0], cov=1)
samples = var2.rvs(100000)
quantiles = var2.cdf(samples)
ax[1].set_title('2D Quantile distribution')
ax[1].hist(quantiles, bins='auto');

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey=False, figsize=(10, 6))

pos1 = np.linspace(-5, 5, 100)
pos2 = np.vstack((x1.flatten(), x2.flatten())).T


ax[0, 0].plot(pos1, var1.pdf(x=pos1))
ax[0, 0].set_title('1D pdf')
ax[0, 1].plot(pos1, var1.cdf(x=pos1))
ax[0, 1].set_title('1D cdf')

x1, x2 = np.meshgrid(pos1, pos1)
ax[1, 0].imshow(var2.pdf(pos2).reshape(100, 100), origin='lower', interpolation='none', 
               extent=[pos1.min(), pos1.max(), pos1.min(), pos1.max()])
ax[1, 0].set_title('2D pdf')
ax[1, 1].imshow(var2.cdf(pos2).reshape(100, 100), origin='lower', interpolation='none', 
               extent=[pos1.min(), pos1.max(), pos1.min(), pos1.max()]);
ax[1, 1].set_title('2D cdf');

In [ ]:
# 1D 
fig, ax = plt.subplots(1, 2, sharex=True, figsize=(10, 3))
var1 = scipy.stats.binom(5, 0.1)
samples = var1.rvs(10000)
pdf = var1.pmf(samples)
quantiles = np.cumsum(pdf) / np.max(np.cumsum(pdf))
ax[0].set_title('1D Quantile distribution')
ax[0].hist(quantiles, bins='auto');
# 2D 
var2 = scipy.stats.multinomial(n=5, p=[0.2, 0.8])
samples = var2.rvs(10000)
pdf = var2.pmf(samples)
quantiles = np.cumsum(pdf) / np.max(np.cumsum(pdf))
ax[1].set_title('2D Quantile distribution')
ax[1].hist(quantiles, bins='auto');